# EDA on Chase Bank Branch Deposits

# Content

This dataset includes a record for every branch of Chase Bank in the United States, including the branch's name and number, date established as a bank office and (if applicable) acquired by JP Morgan Chase, physical location as street address, city, state, zip, and latitude and longitude coordinates, and the amount deposited at the branch (or the institution, for the bank's main office) between July 1 and June 30, 2016, in US dollars.


# Data Source

The location data was scraped from the Chase Bank website. The deposit data was compiled from the Federal Deposit Insurance Corporation's annual Summary of Deposits reports. This perticular data was taken from the Kaggle Link :- https://www.kaggle.com/chasebank/bank-deposits


# EDA Ideas

* Where did Chase Bank customers deposit the most money last year?
* Which bank branch has seen the most growth in deposits?
* How did the bank network of branch locations grow over the past century?
* What city has the most bank branches per capita?

In [263]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt #for graphing
%matplotlib inline
import seaborn as sns #for Plotting
local_path = "C:/Users/Aravind/Desktop/UpX Academy/Data Science Track - 01/PROJECTS TO BE DOING/Project Datasets/Project_Bank/Chase bank-deposits/database.csv"
input_data = pd.read_csv(local_path)                           #load CSV file from local directory
input_data.head()                                         # Preview the first 5 lines of the loaded data

,Institution Name,Main Office,Branch Name,Branch Number,Established Date,Acquired Date,Street Address,City,County,State,Zipcode,Latitude,Longitude,2010 Deposits,2011 Deposits,2012 Deposits,2013 Deposits,2014 Deposits,2015 Deposits,2016 Deposits
0,JPMorgan Chase Bank,1,JPMorgan Chase Bank Main Office,0,01/01/1824,NaN,1111 Polaris Parkway,Columbus,Delaware,OH,43240,40.14453,-82.99115,633131000.0,743268000.0,832455000.0,916543000.0,1.032549e+09,1.069425e+09,1155185000
1,JPMorgan Chase Bank,0,Vernon Hills Scarsdale Branch,2,03/20/1961,NaN,676 White Plains Road,Scarsdale,Westchester,NY,10583,40.97008,-73.80670,293229.0,310791.0,325742.0,327930.0,3.277920e+05,3.414750e+05,381558
2,JPMorgan Chase Bank,0,Great Neck Northern Boulevard Branch,3,09/09/1963,NaN,410 Northern Boulevard,Great Neck,Nassau,NY,11021,40.77944,-73.72240,191011.0,206933.0,216439.0,237983.0,2.341830e+05,2.624550e+05,278940
3,JPMorgan Chase Bank,0,North Hartsdale Branch,4,02/19/1966,NaN,353 North Central Avenue,Hartsdale,Westchester,NY,10530,41.02654,-73.79168,87110.0,88367.0,93163.0,109659.0,1.119850e+05,1.167720e+05,140233
4,JPMorgan Chase Bank,0,Lawrence Rockaway Branch,5,01/16/1965,NaN,335 Rockaway Turnpike,Lawrence,Nassau,NY,11559,40.62715,-73.73675,172608.0,172749.0,189413.0,198445.0,2.051980e+05,2.232000e+05,235594


In [154]:
df= pd.DataFrame(input_data)          
input_data.shape

(5413, 20)

In [155]:
df.isnull().sum()             #Identifying number of Nan Values

Institution Name       0
Main Office            0
Branch Name            0
Branch Number          0
Established Date       0
Acquired Date       1615
Street Address         0
City                   0
County                 0
State                  0
Zipcode                0
Latitude              66
Longitude             66
2010 Deposits        740
2011 Deposits        578
2012 Deposits        329
2013 Deposits        175
2014 Deposits         56
2015 Deposits         19
2016 Deposits          0
dtype: int64

In [156]:
# Since Latitude, Longitude are not required for the purpose of this EDA, we shall remove those columns
df.drop(['Latitude','Longitude'], axis = 1, inplace = True)
df

,Institution Name,Main Office,Branch Name,Branch Number,Established Date,Acquired Date,Street Address,City,County,State,Zipcode,2010 Deposits,2011 Deposits,2012 Deposits,2013 Deposits,2014 Deposits,2015 Deposits,2016 Deposits
0,JPMorgan Chase Bank,1,JPMorgan Chase Bank Main Office,0,01/01/1824,NaN,1111 Polaris Parkway,Columbus,Delaware,OH,43240,633131000.0,743268000.0,832455000.0,916543000.0,1.032549e+09,1.069425e+09,1155185000
1,JPMorgan Chase Bank,0,Vernon Hills Scarsdale Branch,2,03/20/1961,NaN,676 White Plains Road,Scarsdale,Westchester,NY,10583,293229.0,310791.0,325742.0,327930.0,3.277920e+05,3.414750e+05,381558
2,JPMorgan Chase Bank,0,Great Neck Northern Boulevard Branch,3,09/09/1963,NaN,410 Northern Boulevard,Great Neck,Nassau,NY,11021,191011.0,206933.0,216439.0,237983.0,2.341830e+05,2.624550e+05,278940
3,JPMorgan Chase Bank,0,North Hartsdale Branch,4,02/19/1966,NaN,353 North Central Avenue,Hartsdale,Westchester,NY,10530,87110.0,88367.0,93163.0,109659.0,1.119850e+05,1.167720e+05,140233
4,JPMorgan Chase Bank,0,Lawrence Rockaway Branch,5,01/16/1965,NaN,335 Rockaway Turnpike,Lawrence,Nassau,NY,11559,172608.0,172749.0,189413.0,198445.0,2.051980e+05,2.232000e+05,235594
5,JPMorgan Chase Bank,0,Mount Vernon Branch,9,02/25/1964,NaN,22 West First Street,Mount Vernon,Westchester,NY,10550,146820.0,136672.0,140678.0,154313.0,1.487070e+05,1.454990e+05,151949
6,JPMorgan Chase Bank,0,Castle Hill Branch,12,12/11/1965,NaN,784 Castle Hill Avenue,Bronx,Bronx,NY,10473,75131.0,80172.0,82333.0,94538.0,9.704600e+04,1.071660e+05,110592
7,JPMorgan Chase Bank,0,Burnside Avenue Branch,15,02/14/1922,NaN,5 West Burnside Avenue,Bronx,Bronx,NY,10453,39888.0,42546.0,43800.0,45867.0,5.136300e+04,5.669300e+04,62993
8,JPMorgan Chase Bank,0,Gun Hill Branch,16,12/18/1950,NaN,1308 East Gun Hill Road,Bronx,Bronx,NY,10469,73702.0,70049.0,70447.0,78946.0,8.067700e+04,8.275600e+04,88954
9,JPMorgan Chase Bank,0,Kingsbridge Road Branch,18,08/18/1924,NaN,66 East Kingsbridge Road,Bronx,Bronx,NY,10468,69778.0,72816.0,74729.0,75470.0,7.875900e+04,8.397800e+04,88674


In [157]:
df.groupby('Branch Name').size().count()
# Here we can see there are total 5317 branches present.

5317

## *In the year 2016

In [311]:
location = df.groupby(['State', '2016 Deposits']).sum()

In [278]:
deposit_2016 = df.groupby(['2016 Deposits', 'City'])

In [314]:
deposit_2016.sum()

Main Office  Branch Number       Zipcode  \
2016 Deposits City                                                           
0.000000e+00  Abilene            0.000000e+00   3.432000e+03  7.960100e+04   
              Amarillo           0.000000e+00   2.363000e+03  7.910100e+04   
              Amityville         0.000000e+00   1.283000e+03  1.170100e+04   
              Appleton           0.000000e+00   2.897000e+03  5.491100e+04   
              Arlington          0.000000e+00   5.876000e+03  2.280370e+05   
              Atlanta            0.000000e+00   4.408000e+03  3.030500e+04   
              Austin             0.000000e+00   9.234000e+03  2.361970e+05   
              Baton Rouge        0.000000e+00   7.789000e+03  2.124180e+05   
              Beaver Dam         0.000000e+00   2.904000e+03  5.391600e+04   
              Bellaire           0.000000e+00   1.822000e+03  4.390600e+04   
              Boston             0.000000e+00   4.407000e+03  2.110000e+03   
              Bronx              0.000000e+00   2.566000e+03  2.093700e+04   
              Burton             0.000000e+00   3.446000e+03  4.852900e+04   
              Center Moriches    0.000000e+00   1.285000e+03  1.193400e+04   
              Charleston         0.000000e+00   1.811000e+03  2.530100e+04   
              Chicago            0.000000e+00   1.202500e+04  1.212120e+05   
              Clarksburg         0.000000e+00   1.814000e+03  2.630100e+04   
              Columbus           0.000000e+00   8.592000e+03  2.161070e+05   
              Covington          0.000000e+00   2.167000e+03  7.043300e+04   
              Cridersville       0.000000e+00   1.738000e+03  4.580600e+04   
              Dallas             0.000000e+00   2.550200e+04  6.017580e+05   
              Dayton             0.000000e+00   1.749000e+03  4.540200e+04   
              Denison            0.000000e+00   2.362000e+03  7.502000e+04   
              Denver             0.000000e+00   4.404000e+03  8.020200e+04   
              Detroit            0.000000e+00   2.624000e+03  4.822300e+04   
              Dublin             0.000000e+00   1.571000e+03  4.301600e+04   
              Edmond             0.000000e+00   3.318000e+03  7.303400e+04   
              Elk Grove Village  0.000000e+00   3.152000e+03  6.000700e+04   
              Fairfield          0.000000e+00   1.479000e+03  7.004000e+03   
              Farmers Branch     0.000000e+00   1.407000e+03  7.523400e+04   
...                                       ...            ...           ...   
1.293817e+06  New Orleans        0.000000e+00   2.113000e+03  7.017000e+04   
1.449603e+06  New York City      0.000000e+00   7.880000e+02  1.001700e+04   
1.498325e+06  New York City      0.000000e+00   7.940000e+02  1.003800e+04   
1.580901e+06  Seattle            0.000000e+00   6.502000e+03  9.810100e+04   
1.713253e+06  Fort Worth         0.000000e+00   2.326000e+03  7.610200e+04   
1.873614e+06  Louisville         0.000000e+00   3.194000e+03  4.020200e+04   
1.925358e+06  New York City      0.000000e+00   1.235000e+03  1.000500e+04   
1.943526e+06  Austin             0.000000e+00   2.230000e+03  7.870100e+04   
2.072137e+06  New York City      0.000000e+00   1.050000e+02  1.002000e+04   
2.551581e+06  Oklahoma City      0.000000e+00   7.976000e+03  7.310200e+04   
3.230688e+06  Denver             0.000000e+00   3.087000e+03  8.020200e+04   
3.407964e+06  San Francisco      0.000000e+00   4.440000e+03  9.412200e+04   
3.626444e+06  Milwaukee          0.000000e+00   2.852000e+03  5.320200e+04   
3.764612e+06  Garland            0.000000e+00   3.684000e+03  7.504000e+04   
4.328510e+06  Los Angeles        0.000000e+00   4.501000e+03  9.001700e+04   
4.995400e+06  Baton Rouge        0.000000e+00   2.036000e+03  7.080100e+04   
5.164067e+06  Indianapolis       0.000000e+00   2.928000e+03  4.620400e+04   
6.879927e+06  New York City      0.000000e+00   7.078000e+03  1.003600e+04   
7.822138e+06  Phoenix            0.000000e+00   2.414000